# Vergleich verschiedener CNNs

## Imports

In [1]:
import os 
from time import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from matplotlib import pyplot as plt

from ipywidgets import interact,interactive, fixed, interact_manual
import ipywidgets as widgets

import cv2
directory = './GTSRB/Final_Training/Images'
directory_Test = './GTSRB/Final_Test/Images'

## Konvertiere Bilder


Konvertiere die PPM Bilder in JPG

In [12]:
# import os
# rootdir = directory
# extensions = ('.ppm')

# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         ext = os.path.splitext(file)[-1].lower()
#         if ext in extensions:
#             new_extension = ".jpg"
#             path_str = os.path.join(subdir, file)
#             pre, ext = os.path.splitext(path_str)
#             path_renamed = pre + new_extension
#             i = cv2.imread(path_str)
#             cv2.imwrite(path_renamed,i)


## Globale Variablen

In [3]:
num_classes = 43
num_epochs = 40
num_batch_size = 64

## Datenaufbereitung
Laden der Datasets

In [4]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="categorical",
    class_names=0,
    color_mode="rgb",
    batch_size=num_batch_size,
    image_size=(128, 128),
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
)
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=num_batch_size,
    image_size=(128, 128),
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
)



Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


Normalisiere Bilder: 0-255 -> 0-1

In [5]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
norm_train = ds_train.map(lambda x, y: (normalization_layer(x), y))
norm_test = ds_test.map(lambda x, y: (normalization_layer(x), y))
ds_train = norm_train
ds_test = norm_test

## Aufbau der verschiedenen Modelle

Modelle ohne Weights

In [16]:
# res50v2 = tf.keras.applications.ResNet50V2(weights=None, input_shape=(128, 128, 3), classes=num_classes)
# xception = tf.keras.applications.Xception(weights=None, input_shape=(128, 128, 3), classes=num_classes)
# vgg16 = tf.keras.applications.VGG16(weights=None, input_shape=(128, 128, 3), classes=num_classes)
# vgg19 = tf.keras.applications.VGG19(weights=None, input_shape=(128, 128, 3), classes=num_classes)
# mobv2 = tf.keras.applications.MobileNetV2(weights=None, input_shape=(128, 128, 3), classes=num_classes)
# inceptv3 = tf.keras.applications.InceptionV3(weights=None, input_shape=(128, 128, 3), classes=num_classes)
# resnet152v2 = tf.keras.applications.ResNet152V2(weights=None, input_shape=(128, 128, 3), classes=num_classes)

Modelle mit Imagenet Features

In [6]:
res50v2PT = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape=(128, 128, 3),include_top = False, classes=num_classes)
xceptionPT = tf.keras.applications.Xception(weights='imagenet', input_shape=(128, 128, 3),include_top = False, classes=num_classes)
vgg16PT = tf.keras.applications.VGG16(weights='imagenet', input_shape=(128, 128, 3), include_top = False,classes=num_classes)
vgg19PT = tf.keras.applications.VGG19(weights='imagenet', input_shape=(128, 128, 3),include_top = False, classes=num_classes)
mobv2PT = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(128, 128, 3), include_top = False,classes=num_classes)
inceptv3PT = tf.keras.applications.InceptionV3(weights='imagenet', input_shape=(128, 128, 3), include_top = False,classes=num_classes)
resnet152v2PT = tf.keras.applications.ResNet152V2(weights='imagenet', input_shape=(128, 128, 3),include_top = False, classes=num_classes)

Hinzufügen der Klassifizierungs Layer

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)

res50v2PT.trainable = True
res50v2PT = tf.keras.Sequential([
  res50v2PT,
  global_average_layer,
  prediction_layer
])
res50v2PT.add(Activation('softmax'))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)


xceptionPT.trainable = True
xceptionPT = tf.keras.Sequential([
  xceptionPT,
  global_average_layer,
  prediction_layer
])
xceptionPT.add(Activation('softmax'))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)


vgg16PT.trainable = True
vgg16PT = tf.keras.Sequential([
  vgg16PT,
  global_average_layer,
  prediction_layer
])
vgg16PT.add(Activation('softmax'))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)

vgg19PT.trainable = True
vgg19PT = tf.keras.Sequential([
  vgg19PT,
  global_average_layer,
  prediction_layer
])
vgg19PT.add(Activation('softmax'))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)


mobv2PT.trainable = True
mobv2PT = tf.keras.Sequential([
  mobv2PT,
  global_average_layer,
  prediction_layer
])
mobv2PT.add(Activation('softmax'))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)


inceptv3PT.trainable = True
inceptv3PT = tf.keras.Sequential([
  inceptv3PT,
  global_average_layer,
  prediction_layer
])
inceptv3PT.add(Activation('softmax'))

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)


resnet152v2PT.trainable = True
resnet152v2PT = tf.keras.Sequential([
  resnet152v2PT,
  global_average_layer,
  prediction_layer
])
resnet152v2PT.add(Activation('softmax'))

## Kompilieren des Modells
Eine detaillierte Beschreibung der [compile](https://keras.io/api/models/model_training_apis/#compile-method)-Methode findest du in Keras API Referenz.

In [19]:
# res50v2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
# xception.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
# vgg16.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
# vgg19.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
# mobv2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
# inceptv3.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
# resnet152v2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [8]:
res50v2PT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
xceptionPT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
vgg16PT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
vgg19PT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
mobv2PT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
inceptv3PT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
resnet152v2PT.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [20]:
# opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])

In [9]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [1]:
# y_train = []
# for batch in ds_train:
#     for lab in batch[1]:
#         y_train.append(lab.numpy())
# y_train = np.argmax(y_train, axis=1)
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train),
#                                                  y_train)
# class_weights=dict(enumerate(class_weights))

In [2]:
%load_ext tensorboard
%tensorboard --logdir logs

## Training der Modelle

In [26]:
# tensorboard = TensorBoard(log_dir="logs/resnet152v2")
# histRes = resnet152v2.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# resnet152v2.save('resnet152v2.h5')
# del resnet152v2

# tensorboard = TensorBoard(log_dir="logs/res50")
# histRes = res50v2.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# res50v2.save('res50v2.h5')
# del res50v2

# tensorboard = TensorBoard(log_dir="logs/xception")
# histXC = xception.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# xception.save('xception.h5')
# del xception

# tensorboard = TensorBoard(log_dir="logs/vgg16")
# histvgg16 = vgg16.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# vgg16.save('vgg16.h5')
# del vgg16

# tensorboard = TensorBoard(log_dir="logs/vgg19")
# histVgg19 = vgg19.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# vgg19.save('vgg19.h5')
# del vgg19

# tensorboard = TensorBoard(log_dir="logs/mobv2")
# histMobv2 = mobv2.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# mobv2.save('mobv2.h5')
# del mobv2

# tensorboard = TensorBoard(log_dir="logs/inceptv3")
# histInc = inceptv3.fit(ds_train, epochs=num_epochs, validation_data=ds_test, callbacks=[tensorboard])
# inceptv3.save('inceptv3.h5')
# del inceptv3


## Evaluation

In [4]:
# print(np.mean(predictions==ground_truth))
# print(classification_report(ground_truth,predictions))
# print(confusion_matrix(ground_truth,predictions))

## Laden des Test Datensatzes

In [2]:
# Lade das trainierte Modell und teste die Erkennung der Verkehrszeichen-Klassen mit eigenen Beispielen
ds_true_test = tf.keras.preprocessing.image_dataset_from_directory(
    directory_Test,
    labels="inferred",
    label_mode="categorical",
    class_names=0,
    color_mode="rgb",
    batch_size=256,
    image_size=(128, 128),
    shuffle=None,
    seed=1,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
norm_true_test = ds_true_test.map(lambda x, y: (normalization_layer(x), y))
ds_true_test = norm_true_test

Found 12630 files belonging to 1 classes.


In [4]:
x_true_test = []
for batch in ds_true_test:
    for img in batch[0]:
        x_true_test.append(img.numpy())
x_true_test = np.array(x_true_test)

In [5]:
import csv
ground_truth = []
with open('ground_truth.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        for row in csv_reader:
            ground_truth.append(int(row[-1]))
ground_truth = np.array(ground_truth)

# Auswertung der Modelle

## PRETRAINED Weights not locked

In [6]:
start = time()
model = keras.models.load_model('inceptv3PTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("InceptionV3 pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('mobv2PTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("MobileNetV2 pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('res50v2PTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("ResNet50v2 pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('resnet152v2PTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("ResNet152v2 pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('vgg16PTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("VGG16 pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('vgg19PTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("VGG19 pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('xceptionPTTrainable.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("Xception pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

InceptionV3 pretrained Accuracy: 0.969200316706255
Millisekunden Pro Bild: 0.924
MobileNetV2 pretrained Accuracy: 0.9827395091053048
Millisekunden Pro Bild: 0.447
ResNet50v2 pretrained Accuracy: 0.9803642121931908
Millisekunden Pro Bild: 0.785
ResNet152v2 pretrained Accuracy: 0.9704671417260491
Millisekunden Pro Bild: 1.814
VGG16 pretrained Accuracy: 0.9724465558194775
Millisekunden Pro Bild: 0.824
VGG19 pretrained Accuracy: 0.9519398258115598
Millisekunden Pro Bild: 0.848
Xception pretrained Accuracy: 0.9882818685669041
Millisekunden Pro Bild: 0.943


## PRETRAINED Feature-Weights locked

In [6]:
start = time()
model = keras.models.load_model('inceptv3PT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("InceptionV3 pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('mobv2PT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("MobileNetV2 pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('res50v2PT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("ResNet50v2 pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('resnet152v2PT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("ResNet152v2 pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('vgg16PT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("VGG16 pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('vgg19PT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("VGG19 pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('xceptionPT.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("Xception pretrained, weights locked Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))



InceptionV3 pretrained, weights locked Accuracy: 0.6942992874109264
Millisekunden Pro Bild: 0.783
MobileNetV2 pretrained, weights locked Accuracy: 0.7580364212193191
Millisekunden Pro Bild: 0.378
ResNet50v2 pretrained, weights locked Accuracy: 0.7729216152019003
Millisekunden Pro Bild: 0.748
ResNet152v2 pretrained, weights locked Accuracy: 0.755581947743468
Millisekunden Pro Bild: 1.629
VGG16 pretrained, weights locked Accuracy: 0.6688836104513064
Millisekunden Pro Bild: 0.822
VGG19 pretrained, weights locked Accuracy: 0.6038796516231195
Millisekunden Pro Bild: 0.849
Xception pretrained, weights locked Accuracy: 0.7068091844813935
Millisekunden Pro Bild: 0.909


## Not Pretrained

In [6]:
# model = keras.models.load_model('inceptv3.h5')
# predictions_test = model.predict(x_true_test)
# predictions_test = np.argmax(predictions_test, axis=1)
# print("InceptionV3 not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
# print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('mobv2.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("MobileNetV2 not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('res50v2.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("ResNet50v2 not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

# start = time()
# model = keras.models.load_model('resnet152v2.h5')
# predictions_test = model.predict(x_true_test)
# predictions_test = np.argmax(predictions_test, axis=1)
# print("ResNet152v2 not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
# print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('vgg16.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("VGG16 not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('vgg19.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("VGG19 not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))

start = time()
model = keras.models.load_model('xception.h5')
predictions_test = model.predict(x_true_test)
predictions_test = np.argmax(predictions_test, axis=1)
print("Xception not pretrained Accuracy: " + str(np.mean(predictions_test==ground_truth)))
print("Millisekunden Pro Bild: " + str(np.round((time()-start)/len(ground_truth)*1000,3)))


VGG16 not pretrained Accuracy: 0.057007125890736345
Millisekunden Pro Bild: 0.947
VGG19 not pretrained Accuracy: 0.057007125890736345
Millisekunden Pro Bild: 0.896
Xception not pretrained Accuracy: 0.979889152810768
Millisekunden Pro Bild: 0.904


Exporting Model Predictions to CSV

In [17]:
import csv
with open('employee_file.csv') as csv_file:
    with open('output.csv', mode='w') as employee_file:
        employee_writer = csv.writer(employee_file, delimiter=';')
        csv_reader = csv.reader(csv_file, delimiter=';')
        for i,row in enumerate(csv_reader):
            employee_writer.writerow([row[0],predictions_test[i]])